# # Install Libraries and dependencies

In [ ]:
pip install gtts

In [ ]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download("wordnet")
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import re
import PyPDF2
from datetime import datetime

In [ ]:
import nltk
from nltk.corpus import wordnet
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
def extract__section(pdf_path):
    # experience_section = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()

            # match = re.search(r'Experience\s*:?([\s\S]+?)(?:(?:Education)|(?:Skills)|(?:Languages)|(?:Projects)|(?:Certifications)|(?:References)|\Z)', text, re.IGNORECASE)
            # if match:
            #     experience_section += match.group(1).strip() + "\n"
    return text


In [ ]:
def get_exp_sction(text):
  experience_section = ""
  match = re.search(r'Experience\s*:?([\s\S]+?)(?:(?:Education)|(?:Skills)|(?:Languages)|(?:Projects)|(?:Certifications)|(?:References)|\Z)', text, re.IGNORECASE)
  if match:
    experience_section += match.group(1).strip() + "\n"
  return experience_section

In [ ]:
def extract_total_experience(exp_section):
  date_pattern = r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep.(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{4}\s*–\s*(?:Present|\w+\s+\d{4})'
  years = re.findall(date_pattern, exp_section)
  return years

In [ ]:
def format_dates(get_years_list):

  get_current_date = datetime.now()
  # get_current_date = datetime.strptime(get_current_date, "%B %Y")
  # print(datetime.strptime(get_current_date, "%B %Y"))

  modified_list = []

  for item in get_years_list:
      date_parts = item.split(' – ')
      # print(date_parts)
      modified_list.append(date_parts)

  # print(modified_list[0][0].replace('.',''))

  for dt_range in range(len(modified_list)):
    for each_dt in range(len(modified_list[dt_range])):
      get_date = modified_list[dt_range][each_dt].replace('.','')
      if get_date == 'Present' or get_date == 'Current':
        modified_list[dt_range][each_dt] = get_current_date.date()
      else:
        try:
          datetime_obj = datetime.strptime(get_date, "%B %Y")
        except ValueError:
          try:
            datetime_obj = datetime.strptime(get_date, "%b %Y")
          except ValueError:
            print("Error: Unable to parse date string:", get_date)
            continue
        modified_list[dt_range][each_dt] = datetime_obj.date()

      # print(get_date)
  # print(modified_list)
  return modified_list

In [ ]:
def calculate_total_years(formatted_years_list):
  prev_emp_start = datetime.now().date()
  prev_emp_end = datetime.now().date()
  total_days = 0
  total_months = 0
  total_years = 0

  for dt_range in formatted_years_list:
    get_diff = dt_range[1] - dt_range[0]
    # print('\n')
    # print(dt_range[0],dt_range[1])
    # print(prev_emp_start,prev_emp_end)
    # if dt_range[1] <= prev_emp_start:
    #   get_diff = dt_range[1] - dt_range[0]
    # else:
    #   get_diff = prev_emp_start - dt_range[0]

    if get_diff != 0 and get_diff.days > 0:
      total_days += get_diff.days

    prev_emp_start = dt_range[0]
    prev_emp_end = dt_range[1]

  total_years_calc = total_days / 365
  total_years = int(total_years_calc)
  total_months = int((total_years_calc - total_years) * 12)


  return [total_years, total_months]

In [ ]:
def get_exp_year_from_pdf(pdf_path):
  extract_text = extract__section(pdf_path)

  experience_section = get_exp_sction(extract_text)

  years_list = extract_total_experience(experience_section)
  formatted_years_list = format_dates(years_list)
  total_years_worked = calculate_total_years(formatted_years_list)

  return total_years_worked

In [ ]:
def get_skill_section(extract_text):
  skills_section_pattern = r'Skills\s*:?([\s\S]+?)(?:(?:Experience)|(?:Education)|(?:Languages)|(?:Projects)|(?:Certifications)|(?:References)|\Z)'
  skills_section_match = re.search(skills_section_pattern, extract_text, re.IGNORECASE)
  skills_section = None
  if skills_section_match:
      skills_section = skills_section_match.group(1).strip()
      # print("Skills Section:")
      # print(skills_section)
  else:
      print("Skills section not found.")

  return skills_section


In [ ]:
def create_skill_dict(get_all_skills):
  all_skills = get_all_skills.split('\n')
  skill_list = {}
  for s1 in all_skills:
    new_s1 = s1.split(':')
    for n1 in new_s1[1].split(','):
      skill_list[n1] = 0
  # print(skill_list)

  return skill_list

In [ ]:
def find_skill_usage(skill_dict, extract_text):
  for each_skill in skill_dict:
    skill_search = extract_text.count(each_skill)
    skill_dict[each_skill] = skill_search
  # print(skill_list)
  return skill_dict

In [ ]:
def get_skills_section_from_pdf(pdf_path):
  extract_text = extract__section(pdf_path)
  get_all_skills = get_skill_section(extract_text)
  skill_dict = create_skill_dict(get_all_skills)
  updated_skill_dict = find_skill_usage(skill_dict, extract_text)

  return updated_skill_dict

# Input

In [ ]:
pdf_path = "Jake_s_Resume__9_.pdf"
# pdf_path = "jakes-resume.pdf"


candidate_exp_years, candidate_exp_months = get_exp_year_from_pdf(pdf_path)
get_years_of_exp = str(candidate_exp_years) + 'years '+ str(candidate_exp_months)+ 'months '
# print(candidate_exp_years, 'years ', candidate_exp_months, 'months ')
print("Years of Experience: ", get_years_of_exp)

get_skills = get_skills_section_from_pdf(pdf_path)

print('\nAreas of Expertise:', get_skills.keys())

Years of Experience:  3years 2months 

Areas of Expertise: dict_keys([' Java', ' Python', ' C/C++', ' SQL (Postgres)', ' JavaScript', ' HTML/CSS', ' R', ' React', ' Node.js', ' Flask', ' JUnit', ' WordPress', ' Material-UI', ' FastAPI', ' Git', ' Docker', ' TravisCI', ' Google Cloud Platform', ' VS Code', ' Visual Studio', ' PyCharm', ' IntelliJ', ' Eclipse', ' pandas', ' NumPy', ' Matplotlib'])


# Chat

In [ ]:
def get_professional_words(term):
    professional_words = set()
    for synset in wordnet.synsets(term):
        for lemma in synset.lemmas():
            synonym = lemma.name()
            professional_words.add(synonym)
            for hypernym in synset.hypernyms():
                professional_words.add(hypernym.lemma_names()[0])
            for hyponym in synset.hyponyms():
                professional_words.add(hyponym.lemma_names()[0])
    return professional_words

In [ ]:
def get_professional_vocab_exp():
  expereince_terms = ["professional", "experience", "career", "occupation", "work", "job", "employment", "vocation", "involve"]
  vocab_exp = set()
  for term in expereince_terms:
      vocab_exp.update(get_professional_words(term))
  # print(vocab_exp)
  return vocab_exp

In [ ]:
def get_professional_vocab_skill():
  skill_terms = ['skill', 'expertise', 'knowledge']
  vocab_skill = set()
  for term in skill_terms:
      vocab_skill.update(get_professional_words(term))
  # print(vocab_skill)
  return vocab_skill

In [ ]:
def check_required_info(required_list):
  professional_vocab_exp = get_professional_vocab_exp()
  professional_vocab_skill = get_professional_vocab_skill()

  for word in required_list:
    if word in professional_vocab_exp and word in professional_vocab_skill:
      return [get_years_of_exp, get_skills]
    elif word in professional_vocab_exp:
      return [get_years_of_exp, None]
    elif word in professional_vocab_skill:
      return [None, get_skills]
    else:
      continue
  return [None, None]

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    preprocessed_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum() and token not in stop_words]
    return preprocessed_tokens


def provide_answer(input_question):
    preprocessed_question = preprocess_text(input_question)

    lemmatized_words = []
    for word in preprocessed_question:
        pos_tag = nltk.pos_tag([word])[0][1][0].lower()
        if pos_tag in ['a', 'r', 'n', 'v']:
            lemma = lemmatizer.lemmatize(word, pos=pos_tag)
        else:
            lemma = lemmatizer.lemmatize(word)
        lemmatized_words.append(lemma)

    # print(lemmatized_words)

    return check_required_info(lemmatized_words)



# Chat: Ask Question

In [ ]:
question = input("Please enter your question: ")

Please enter your question: How many years of experience?


In [ ]:

# input_question = "What is the extent of your professional experience in your current domain?"


answer_exp, answer_skill = provide_answer(question)

print(question)
print("Answer:")

if not answer_exp and  not answer_skill:
  print("Sorry, couldn't extract the requested information from the CV.")

if answer_exp:
  print(answer_exp)

if answer_skill:
  print('Expertise')
  for k_skill in answer_skill:
    if answer_skill[k_skill] > 1:
      print(k_skill)
  print('Familiar')
  for k_skill in answer_skill:
    if answer_skill[k_skill] <= 1:
      print(k_skill)



How many years of experience?
Answer:
3years 2months 


In [ ]:
from gtts import gTTS
import os

In [ ]:
def text_to_speech(text, lang='en'):

    tts = gTTS(text=text, lang=lang)


    tts.save("output.mp3")


    os.system("start output.mp3")

if not answer_exp and  not answer_skill:
  text_to_speech("Sorry, couldn't extract the requested information from the CV.")

if answer_exp:
  text_to_speech(answer_exp)

if answer_skill:
  text_to_speech(answer_skill)

In [ ]:
# def text_to_speech(text, lang='en'):

#     tts = gTTS(text=text, lang=lang)


#     tts.save("output.mp3")


#     os.system("start output.mp3")

# if not answer_exp and  not answer_skill:
#   text_to_speech("Sorry, couldn't extract the requested information from the CV.")

# if answer_exp:
#   text_to_speech(answer_exp)

# if answer_skill:
#   text_to_speech(answer_skill)